In [47]:
import mcpi.minecraft as minecraft
import mcpi.block as block
import mcpi.entity as entity
from mcpi.vec3 import Vec3
import numpy as np
import random
import pickle
import glob
import time
import os
from PIL import Image

DIR_X1 = 1
DIR_X0 = 2
DIR_Z1 = 3
DIR_Z0 = 4
DIR_UP = 5

In [23]:
def load_room(path):
    with open(path, 'rb') as f:
        spec = pickle.load(f)
        return Room(
            name=spec['name'],
            side=spec['side'],
            door_style=spec['door_style'],
            source_location=spec['source_location'],
            version=spec['version'],
            blocks=spec['blocks'],
        )

class Room:
    def __init__(self, name, side, door_style, source_location, version, blocks):
        self.name = name
        self.side = side
        self.door_style = door_style
        self.source_location = source_location
        self.version = version
        self.blocks = blocks

    def draw(self, x, y, z):
        seconds = [
            block.TORCH[0],
            block.REDSTONE_TORCH_INACTIVE[0],
            block.REDSTONE_TORCH_ACTIVE[0],
            block.DOOR_ACACIA[0],
            block.DOOR_BIRCH[0],
            block.DOOR_DARK_OAK[0],
            block.DOOR_IRON[0],
            block.DOOR_JUNGLE[0],
            block.DOOR_SPRUCE[0],
            block.DOOR_WOOD[0],
            (block.SIGN())[0],
            block.CARPET[0],
        ]

        i = 0
        for y in np.arange(0, self.side, 1):
            for x in np.arange(0, self.side, 1):
                for z in np.arange(0, self.side, 1):
                    #mc.setBlockWithNBT(0.0+x, 0.0+y, 0.0+z, self.blocks[i])
                    #i += 1
                    b = self.blocks[i]
                    if b[0] not in seconds:
                        mc.setBlockWithNBT(0.0+x, 0.0+y, 0.0+z, b)
                    i += 1
        
        i = 0
        for y in np.arange(0, self.side, 1):
            for x in np.arange(0, self.side, 1):
                for z in np.arange(0, self.side, 1):
                    b = self.blocks[i]
                    if b[0] in seconds:
                        mc.setBlockWithNBT(0.0+x, 0.0+y, 0.0+z, b)
                    i += 1

    def get_block_at(self,x,y,z):
        return self.blocks[y*self.side*self.side+x*self.side+z]
    
    def get_image_at_y(self, y=1):
        pixels = []
        for x in np.arange(0, self.side, 1):
            for z in np.arange(0, self.side, 1):
                px = self.get_block_at(x,y,z).getRGBA()
                pixels.append([px[0], px[1], px[2]])

        pixels = np.array(pixels, dtype=np.uint8).reshape((9,9,3))
        return Image.fromarray(pixels)

    def write(self, dir='rooms'):
        with open('%s/%s.pickle' % (dir, r), 'wb') as f:
            pickle.dump({
                'name': self.name,
                'side': self.side,
                'door_style': self.door_style,
                'source_location': self.source_location,
                'version': self.version,
                'blocks': self.blocks,
            }, f)

63

In [12]:
mc = minecraft.Minecraft.create(address='192.168.1.100')

In [28]:
side = 9
p = mc.player.getPos()
p.y = p.y-1
for x in np.arange(0, side, 1):
    for y in np.arange(0, side, 1):
        for z in np.arange(0, side, 1):
            if x==0 or x==side-1 or y==0 or y==side-1 or z==0 or z==side-1:
                mc.setBlock(p.x+x, p.y+y, p.z+z, block.STONE)
                


In [14]:
#print(p)
coords = {}
coords['bathroom'] = Vec3(171,0,158)
coords['dining'] = Vec3(319,-1,137)
coords['library'] = Vec3(336, -1, 96)
coords['factory'] = Vec3(366, -1, 75)
coords['clothes_shop'] = Vec3(399, -1, 48)
coords['swimming_pool'] = Vec3(377, -1, -33)
coords['school'] = Vec3(364, -1, -113)
coords['lazerforce'] = Vec3(359, -1, -101)
coords['train_station'] = Vec3(341, -1, -64)
coords['castle'] = Vec3(260, -1, -92)
coords['spider_room'] = Vec3(240, -1, -50)
coords['sticker_room'] = Vec3(287, -1, -37)
coords['castle_dungeon'] = Vec3(267, -1, -1)
coords['war_room'] = Vec3(414, -1, -73)
coords['bank'] = Vec3(439, -1, -121)
coords['pokemon_room'] = Vec3(454, -1, -84)
coords['dice_room'] = Vec3(415, -1, -105)
coords['miners_cave'] = Vec3(387, -1, -74)

In [47]:
for r in coords.keys():
    print('Processing %s...' % r)
    side = 9
    blocks = mc.getBlocksWithNBT(coords[r].x, coords[r].y, coords[r].z, coords[r].x+side-1, coords[r].y+side-1, coords[r].z+side-1)
    room_obj = Room(
        name=r,
        side=9,
        door_style='middle',
        source_location=coords[r],
        version=1,
        blocks=blocks,
    )
    with open('rooms/%s.pickle' % r, 'wb') as f:
        room_obj.write()



Processing bathroom...
Processing dining...
Processing library...
Processing factory...
Processing clothes_shop...
Processing swimming_pool...
Processing school...
Processing lazerforce...
Processing train_station...
Processing castle...
Processing spider_room...
Processing sticker_room...
Processing castle_dungeon...
Processing war_room...
Processing bank...
Processing pokemon_room...
Processing dice_room...
Processing miners_cave...


In [24]:
rooms = {}
for rf in glob.glob("rooms/*.pickle"):
    room = load_room(rf)
    rooms[room.name] = room


In [49]:
for r in rooms.keys():
    rooms[r].get_image_at_y().save('rooms_images/%s.png' % r)

In [28]:
rooms['dining'].draw(0,0,0)

In [64]:
cp = mc.player.getPos()
mc.player.setPos(-1, 0, -1)


In [46]:
mc.player.setPos(cp)

In [129]:
i = 0
for y in np.arange(0, side, 1):
    for x in np.arange(0, side, 1):
        for z in np.arange(0, side, 1):
            mc.setBlockWithNBT(0.0+x, 0.0+y, 0.0+z, rooms['factory']['blocks'][i])
            i += 1

In [57]:
def corridor_maker(cx,cy,cz,holes=[]):
    side = 9
    is_hole = {
        'x0': lambda x,y,z: x==0 and y!=0 and y!=side-1 and z!=0 and z!=side-1,
        'x1': lambda x,y,z: x==side-1 and y!=0 and y!=side-1 and z!=0 and z!=side-1,
        'y0': lambda x,y,z: y==0 and x!=0 and x!=side-1 and z!=0 and z!=side-1,
        'y1': lambda x,y,z: y==side-1 and x!=0 and x!=side-1 and z!=0 and z!=side-1,
        'z0': lambda x,y,z: z==0 and y!=0 and y!=side-1 and x!=0 and x!=side-1,
        'z1': lambda x,y,z: z==side-1 and y!=0 and y!=side-1 and x!=0 and x!=side-1,
    }

    for x in np.arange(0, side, 1):
        for y in np.arange(0, side, 1):
            for z in np.arange(0, side, 1):
                mc.setBlock(cx+x, cy+y, cz+z, block.AIR)

    for x in np.arange(0, side, 1):
        for y in np.arange(0, side, 1):
            for z in np.arange(0, side, 1):
                wall = False
                if x==0 or x==side-1 or y==0 or y==side-1 or z==0 or z==side-1:
                    wall = True
                for hole in holes:
                    if is_hole[hole](x,y,z):
                        wall = False
                if wall:
                    mc.setBlock(cx+x, cy+y, cz+z, block.STONE_BRICK)
                else:
                    if y==side-2 and random.randint(1, 4)==1:
                        mc.setBlock(cx+x, cy+y, cz+z, block.COBWEB)
                    else:
                        mc.setBlock(cx+x, cy+y, cz+z, block.AIR)

    torch_wall = set(set(is_hole.keys()).difference(set(holes))).difference(['y1']).pop()
    print(torch_wall)
    if torch_wall=='x0':
        mc.setBlock(0.0+1, 0.0+side/2, 0.0+side/2, block.TORCH[0], DIR_X1)
    if torch_wall=='x1':
        mc.setBlock(0.0+side-2, 0.0+side/2, 0.0+side/2, block.TORCH[0], DIR_X0)
    if torch_wall=='z0':
        mc.setBlock(0.0+side/2, 0.0+side/2, 0.0+1, block.TORCH[0], DIR_Z1)
    if torch_wall=='z1':
        mc.setBlock(0.0+side/2, 0.0+side/2, 0.0+side-2, block.TORCH[0], DIR_Z0)
    if torch_wall=='y0':
        mc.setBlock(0.0+side/2, 0.0+1, 0.0+side/2, block.STONE_BRICK_CHISELED)
        mc.setBlock(0.0+side/2, 0.0+2, 0.0+side/2, block.TORCH[0], DIR_UP)

                    


y0
